# **Lorcana Card Prediction**
By Aarohi Chopra and Daniel Quintana

This project is about Lorcana cards and trying to predict certain things about them given the model and decks.




## Imports

In [ ]:
!pip install plotly
!pip install dash

In [ ]:
import requests
import pandas as pd
import numpy as np
import random
import plotly.express as px
import plotly.graph_objs as go
from plotly.graph_objects import Figure, Box
from dash import Dash, dcc, html, Input, Output, no_update, callback
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

# Data Preprocessing

Here we are querying a public API for the lorcana cards

In [ ]:
# API endpoint for fetching all cards
url = "https://api.lorcana-api.com/cards/all"

# Making a GET request to the API
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Convert the JSON response to a pandas DataFrame
    data = response.json()
    df = pd.DataFrame(data)

    # Ensure lore, strength, and willpower columns contain numerical values
    numeric_cols = ['Lore', 'Strength', 'Willpower']
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce').fillna(0)

    # Print the DataFrame

else:
    print("Failed to fetch data. Status code:", response.status_code)

df

,Artist,Set_Name,Classifications,Abilities,Set_Num,Color,Franchise,Image,Cost,Inkable,Name,Type,Lore,Rarity,Unique_ID,Card_Num,Body_Text,Willpower,Strength,Set_ID,Flavor_Text,Card_Variants,Move_Cost
0,Cam Kendell,Into the Inklands,"Dreamborn, Hero",Bodyguard,3,Amber,,https://lorcana-api.com/images/baloo/von_bruinwald_xiii/baloo-von_bruinwald_xiii-large.png,3,False,Baloo - von Bruinwald XIII,Character,1.0,Rare,INK-001,1,"Bodyguard (This character may enter play exerted. An opposing character who challenges one of your characters must choose one with Bodyguard if able.)\n\nLet's Make Like A Tree - When this character is banished, gain 2 lore.",3.0,0.0,INK,NaN,NaN,NaN
1,Jacob McAlister,Into the Inklands,"Storyborn, Hero",NaN,3,Amber,,https://lorcana-api.com/images/bernard/brand-new_agent/bernard-brand-new_agent-large.png,4,True,Bernard - Brand-New Agent,Character,2.0,Rare,INK-002,2,"I'll Check It Out - At the end of your turn, if this character is exerted, you may ready another chosen character of yours.",5.0,1.0,INK,"""You stay there. I'll look for scattered lore.""",NaN,NaN
2,"Evana Kisa, Jochem van Gool",Into the Inklands,"Storyborn, Villain",NaN,3,Amber,,https://lorcana-api.com/images/chernabog/evildoer/chernabog-evildoer-large.png,10,False,Chernabog - Evildoer,Character,3.0,Super Rare,INK-003,3,"The Power Of Evil - For each character card in your discard, you pay 1{i} less to play this character.\n\nSummon The Spirits - When you play this character, shuffle all character cards from your discard into your deck.",9.0,9.0,INK,Darkness calls to minions everywhere,"(enchanted, placeholder)",NaN
3,Kapik,Into the Inklands,"Storyborn, Puppy",NaN,3,Amber,,https://lorcana-api.com/images/dalmatian_puppy/tail_wagger/dalmatian_puppy-tail_wagger-large.png,2,True,Dalmatian Puppy - Tail Wagger,Character,1.0,Common,INK-004,4,Where Did They All Come From? - You may have up to 99 copies of Dalmatian Puppy - Tail Wagger in your deck.,3.0,2.0,INK,First they steal your heart. Then they steal your chair.,"(card_num:4b, placeholder),\n(card_num:4c, placeholder),\n(card_num:4d, placeholder),\n(card_num:4e, placeholder)",NaN
4,Jeanne Plounevez,Into the Inklands,"Storyborn, Ally",Bodyguard,3,Amber,,https://lorcana-api.com/images/joshua_sweet/the_doctor/joshua_sweet-the_doctor-large.png,4,True,Joshua Sweet - The Doctor,Character,2.0,Common,INK-005,5,Bodyguard (This character may enter play exerted. An opposing character who challenges one of your characters must choose one with Bodyguard if able.),5.0,1.0,INK,"""Heading out into the Inklands? Come on back if you need patching up.""",NaN,NaN
5,Nicoletta Baldari,Into the Inklands,"Storyborn, Hero, Princess",NaN,3,Amber,,https://lorcana-api.com/images/kida/atlantean/kida-atlantean-large.png,1,True,Kida - Atlantean,Character,1.0,Common,INK-006,6,NaN,2.0,2.0,INK,Welcome to the Inklands.,NaN,NaN
6,Adam Fenton,Into the Inklands,"Floodborn, Hero, Princess",Shift 3,3,Amber,,https://lorcana-api.com/images/kida/protector_of_atlantis/kida-protector_of_atlantis-large.png,5,True,Kida - Protector of Atlantis,Character,2.0,Legendary,INK-007,7,"Shift 3 (You may pay 3{i} to play this on top of one of your characters named Kida.)\n\nPerhaps We Can Save Our Future - When you play this character, all characters get -3{s} until the start of your next turn.",5.0,3.0,INK,"""She has been chosen."" - King Kashekim Nedakh","(enchanted, placeholder)",NaN
7,Brian Weisz,Into the Inklands,"Storyborn, Puppy",NaN,3,Amber,,https://lorcana-api.com/images/lucky/the_15th_puppy/lucky-the_15th_puppy-large.png,4,False,Lucky - The 15th Puppy,Character,1.0,Rare,INK-008,8,"Good As New - {e}: Reveal the top 3 cards of your deck. You may put each character card with cost 2 or less into your hand. Put the rest on the bottom of your deck in any order.\n\nPuppy Love - Whenever this character quests, if you have 4 or more other characters in play, your other characters get +1{l} this turn.",3.0,2.0,INK,NaN,NaN,NaN
8,Erika Wiseman,Into the Inklands,"D

Here we are cleaning up our dataset. We drop some columns that aren't relavent and clean up some that we will use later
Dropped colums:
Flavor_Text

1.   Set_ID
2.   Set_Name
3.   Artist
4.   Rarity
5.   Card_Variants




In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

df_cards = df.copy()
# 1. Drop specified columns
df_cards.drop(['Flavor_Text','Set_ID', 'Set_Name', 'Artist', 'Rarity', 'Card_Variants'], axis=1, inplace=True)

# 2. Rename 'Color' column to 'Ink'
df_cards.rename(columns={'Color': 'Ink'}, inplace=True)

# 3. Create 'card_id' column by concatenating 'Set_Num' and 'Card_Num'
df_cards['Card_Id'] = df_cards['Set_Num'].astype(str) + '_' + df_cards['Card_Num'].astype(str)

# 4. Drop the 'Set_Num' and 'Card_Num' columns
df_cards.drop(columns=['Set_Num', 'Card_Num'], inplace=True)

# 5. Rename 'Abilities' column to 'KeywordAbilities'
df_cards.rename(columns={'Abilities': 'KeywordAbilities'}, inplace=True)

# 6. Add a new, empty 'Abilities' column
df_cards['Abilities'] = ''

# 7. Set 'card_id' as the new index of the DataFrame
df_cards.set_index('Card_Id', inplace=True)
df_cards.sort_index(inplace=True)

# 8. Display the first few rows of the DataFrame
df_cards

,Classifications,KeywordAbilities,Ink,Franchise,Image,Cost,Inkable,Name,Type,Lore,Unique_ID,Body_Text,Willpower,Strength,Move_Cost,Abilities
Card_Id,,,,,,,,,,,,,,,,
1_1,"Storyborn, Hero, Princess",NaN,Amber,,https://lorcana-api.com/images/ariel/on_human_legs/ariel-on_human_legs-large.png,4,True,Ariel - On Human Legs,Character,2.0,TFC-001,Voiceless - This character can't {e} to sing songs.,4.0,3.0,NaN,
1_10,"Storyborn, Ally","Bodyguard, Support",Amber,,https://lorcana-api.com/images/maximus/palace_horse/maximus-palace_horse-large.png,5,True,Maximus - Palace Horse,Character,1.0,TFC-010,"Bodyguard (This character may enter play exerted. An opposing character who challenges one of your characters must choose one with Bodyguard if able.)\n\nSupport (Whenever this character quests, you may add their {s} to another chosen character's {s} this turn.)",5.0,4.0,NaN,
1_100,NaN,NaN,Emerald,,https://lorcana-api.com/images/vicious_betrayal/vicious_betrayal-large.png,1,True,Vicious Betrayal,Action,0.0,TFC-100,"Chosen character gets +2 {s} this turn. If a villain character is chosen, they get +3 {s} instead.",0.0,0.0,NaN,
1_101,NaN,NaN,Emerald,,https://lorcana-api.com/images/dr._facilier's_cards/dr._facilier's_cards-large.png,2,False,Dr. Facilier's Cards,Item,0.0,TFC-101,The Cards Will Tell - {e} - You pay 1 {i} less for the next action you play this turn.,0.0,0.0,NaN,
1_102,NaN,NaN,Emerald,,https://lorcana-api.com/images/stolen_scimitar/stolen_scimitar-large.png,2,True,Stolen Scimitar,Item,0.0,TFC-102,"Slash - {e} - Chosen character gets +1 {s} this turn. If a character named Aladdin is chosen, he get +2 {s} instead.",0.0,0.0,NaN,
1_103,"Storyborn, Ally",NaN,Ruby,,https://lorcana-api.com/images/abu/mischievous_monkey/abu-mischievous_monkey-large.png,3,True,Abu - Mischievous Monkey,Character,2.0,TFC-103,NaN,2.0,3.0,NaN,
1_104,"Floodborn, Hero",Shift 5,Ruby,,https://lorcana-api.com/images/aladdin/heroic_outlaw/aladdin-heroic_outlaw-large.png,7,True,Aladdin - Heroic Outlaw,Character,2.0,TFC-104,"Shift 5 (You may pay 5 {i} to play this on top of one of your characters named Aladdin.) \r \r Daring Exploit - During your turn, whenever this character banishes another character in a challenge, you gain 2 lore and each opponent loses 2 lore.",5.0,5.0,NaN,
1_105,"Storyborn, Hero",NaN,Ruby,,https://lorcana-api.com/images/aladdin/street_rat/aladdin-street_rat-large.png,3,True,Aladdin - Street Rat,Character,1.0,TFC-105,"Improvise - When you play this character, each opponent loses 1 lore.",2.0,2.0,NaN,
1_106,"Storyborn, Ally, Captain",NaN,Ruby,,https://lorcana-api.com/images/captain/colonel's_lieutenant/captain-colonel's_lieutenant-large.png,5,True,Captain - Colonel's Lieutenant,Character,1.0,TFC-106,NaN,5.0,6.0,NaN,


Due to being a publicly maintained dataset there are some issues with the formatting of certain card text and inconsistencies, so we fix those here!

In [ ]:
# Updates to apply
updates = {
    #"2_197": "Resist +2 (Damage dealt to this character is reduced by 2.)\n\nWhat You Give Is What You Get: While this character is exerted and you have no cards in your hand, opponents can't play actions.",
    "2_195": "Bodyguard (This character may enter play exerted. An opposing character who challenges one of your characters must choose one with Bodyguard if able.)\n\nResist +1 (Damage dealt to this character is reduced by 1.)",
    "2_22": "Ah Choo! - Whenever you play this character or another Seven Dwarfs character, you may give chosen character -1 {s} this turn.",
    "2_193": "Skirmish - {e}  Deal 1 damage to chosen character.",
    "2_196": "Resist +2 (Damage dealt to this character is reduced by 2.)\n\nWhat You Give Is What You Get: While this character is exerted and you have no cards in your hand, opponents can't play actions.",
    "2_191": "Resist +1 (Damage dealt to this character is reduced by 1.)\n\nScout Leader: During your turn, whenever this character banishes another character in a challenge, you may deal 2 damage to chosen character. ",
    "2_189": "Bodyguard (This character may enter play exerted. An opposing character who challenges one of your characters must choose one with Bodyguard if able.)\n\nBlades: This character can challenge ready characters.",
    "2_185": "Resist +1 (Damage dealt to this character is reduced by 1.)\n\nScout Leader: During your turn, whenever this character banishes another character in a challenge, you may deal 2 damage to chosen character.",
    "2_183": "Shift 2 (You may pay 2 {i} to play this on top of one of your characters named Jafar.)\n\nNow Where Were We?: During your turn, whenever this character banishes another character in a challenge, you may draw a card.",
    "2_181": "Shift 4 (You may pay 4 {i} to play this on top of one of your characters named Hercules.)\n\nResist +2 (Damage dealth to this character is reduced by 2.)",
    "1_175": "Shift 3 (You may pay 3 {i} to play this on top of one of your characters named Captain Hook.) \n\nChallenger +3 (When challenging, this character get +3 {s}.) \n\nStolen Dust - Characters with cost 3 or less can't challenge this character.",
    "3_192": "Step Down Or Fight - When you play this character and whenever he banishes another character in a challenge during your turn, you may choose one. Draw 2 cards, then choose and discard 2 cards. Deal 2 damage 2 chosen character."
}

# Apply updates
for index, text in updates.items():
    df_cards.at[index, 'Body_Text'] = text

1. This extracts the Key Abilities that cards have and adds them to a seperate thing

2. Cleans it a bit

3. Does pattern matching

4. Finally to character cards adds their ability to the data(Ability column)

In [ ]:
import pandas as pd
import re  # Import regular expressions library

def extract_abilities(body_text):
    # Initialize the list of abilities
    abilities = []

    # Check if body_text is a string (e.g., not NaN)
    if not isinstance(body_text, str):
        return abilities  # Return empty list if not a string

    # Normalize line breaks (replace \r with \n)
    body_text = body_text.replace('\r', '\n')

    # Define keywords and patterns to look for
    keywords = ['Bodyguard', 'Evasive', 'Ward', 'Support', 'Rush', 'Reckless', 'Improvise']
    patterns = [
        r'\bShift (\d+)',  # Correctly matches 'Shift' followed by a number
        r'\bResist (\d+)',  # Correctly matches 'Resist' followed by a number
        r'\bChallenger (\d+)'  # Correctly matches 'Challenger' followed by a number
    ]

    # Remove text within parentheses to clean up descriptions before matching abilities
    clean_text = re.sub(r'\(.*?\)', '', body_text)

    # Process text to ensure keywords only match at the start of a line or text
    for keyword in keywords:
        pattern = r'(^|\n){}(\s|\()'.format(keyword)
        if re.search(pattern, clean_text):
            abilities.append(keyword)

    # Check for patterns with numbers
    for pattern in patterns:
        matches = re.findall(pattern, clean_text)
        for match in matches:
            ability = pattern.split(r'\b')[1].split(' ')[0] + " " + match  # Extract the ability name and append the number
            if ability not in abilities:
                abilities.append(ability)

    # Process each line separately to extract abilities based on - or :
    for line in clean_text.split('\n'):
        if line:  # Ensure the line isn't empty
            first_part = line.split(' - ')[0].split(': ')[0].strip()
            if first_part and first_part not in abilities:  # Avoid duplicates
                abilities.append(first_part)

    # Remove duplicates and keep order
    abilities = list(dict.fromkeys(abilities))
    return abilities

# Apply the function to a filtered DataFrame based on Type and create a new 'Abilities' column
filtered_df = df_cards[df_cards['Type'].isin(['Character'])].copy()
filtered_df['Abilities'] = filtered_df['Body_Text'].apply(extract_abilities)

# Display the first few rows of the filtered DataFrame to verify
filtered_df

#filtered_df.loc[['2_197','2_195','2_22', '2_193', '2_196', '2_191', '2_189', '2_185', '2_183', '2_181', '1_175', '3_192']]

,Classifications,KeywordAbilities,Ink,Franchise,Image,Cost,Inkable,Name,Type,Lore,Unique_ID,Body_Text,Willpower,Strength,Move_Cost,Abilities
Card_Id,,,,,,,,,,,,,,,,
1_1,"Storyborn, Hero, Princess",NaN,Amber,,https://lorcana-api.com/images/ariel/on_human_legs/ariel-on_human_legs-large.png,4,True,Ariel - On Human Legs,Character,2.0,TFC-001,Voiceless - This character can't {e} to sing songs.,4.0,3.0,NaN,[Voiceless]
1_10,"Storyborn, Ally","Bodyguard, Support",Amber,,https://lorcana-api.com/images/maximus/palace_horse/maximus-palace_horse-large.png,5,True,Maximus - Palace Horse,Character,1.0,TFC-010,"Bodyguard (This character may enter play exerted. An opposing character who challenges one of your characters must choose one with Bodyguard if able.)\n\nSupport (Whenever this character quests, you may add their {s} to another chosen character's {s} this turn.)",5.0,4.0,NaN,"[Bodyguard, Support]"
1_103,"Storyborn, Ally",NaN,Ruby,,https://lorcana-api.com/images/abu/mischievous_monkey/abu-mischievous_monkey-large.png,3,True,Abu - Mischievous Monkey,Character,2.0,TFC-103,NaN,2.0,3.0,NaN,[]
1_104,"Floodborn, Hero",Shift 5,Ruby,,https://lorcana-api.com/images/aladdin/heroic_outlaw/aladdin-heroic_outlaw-large.png,7,True,Aladdin - Heroic Outlaw,Character,2.0,TFC-104,"Shift 5 (You may pay 5 {i} to play this on top of one of your characters named Aladdin.) \r \r Daring Exploit - During your turn, whenever this character banishes another character in a challenge, you gain 2 lore and each opponent loses 2 lore.",5.0,5.0,NaN,"[Shift 5, Daring Exploit]"
1_105,"Storyborn, Hero",NaN,Ruby,,https://lorcana-api.com/images/aladdin/street_rat/aladdin-street_rat-large.png,3,True,Aladdin - Street Rat,Character,1.0,TFC-105,"Improvise - When you play this character, each opponent loses 1 lore.",2.0,2.0,NaN,[Improvise]
1_106,"Storyborn, Ally, Captain",NaN,Ruby,,https://lorcana-api.com/images/captain/colonel's_lieutenant/captain-colonel's_lieutenant-large.png,5,True,Captain - Colonel's Lieutenant,Character,1.0,TFC-106,NaN,5.0,6.0,NaN,[]
1_107,"Storyborn, Villain, Pirate, Captain","Rush, Evasive, Reckless",Ruby,,https://lorcana-api.com/images/captain_hook/ruthless_pirate/captain_hook-ruthless_pirate-large.png,7,False,Captain Hook - Ruthless Pirate,Character,2.0,TFC-107,"Rush (This character can challenge the turn they're played.) \n\nYou Coward! - While this character is exerted, opposing characters with Evasive gain Reckless. (They can't quest and must challenge if able.)",5.0,5.0,NaN,"[Rush, You Coward!]"
1_108,Storyborn,NaN,Ruby,,https://lorcana-api.com/images/donald_duck/boisterous_fowl/donald_duck-boisterous_fowl-large.png,2,True,Donald Duck - Boisterous Fowl,Character,1.0,TFC-108,NaN,3.0,2.0,NaN,[]
1_109,"Dreamborn, Hero, Queen, Sorcerer",NaN,Ruby,,https://lorcana-api.com/images/elsa/ice_surfer/elsa-ice_surfer-large.png,4,True,Elsa - Ice Surfer,Character,1.0,TFC-109,"That's No Blizzard! - Whenever you play a character named Anna, ready this character. This character can't quest for the rest of this turn.",4.0,3.0,NaN,[That's No Blizzard!]


Sample code to see if plotting cards with hover over images works, it does!!!!!

In [ ]:
# Assuming filtered_df has columns 'Cost', 'Strength', 'Name', 'Image', 'Body_Text'
fig = go.Figure(data=[
    go.Scatter(
        x=filtered_df["Cost"],
        y=filtered_df["Strength"],
        mode='markers',
        marker=dict(size=10, color=filtered_df["Cost"]),
        text=filtered_df["Name"],  # This will be displayed as hover info
        customdata=filtered_df[["Image", "Body_Text"]]
    )
])

# Turn off default Plotly hover effect
fig.update_traces(hoverinfo="none", hovertemplate=None)

# Set the layout for the plot
fig.update_layout(
    xaxis=dict(title='Cost'),
    yaxis=dict(title='Strength'),
    plot_bgcolor='rgba(255,255,255,0.1)'
)

app = Dash(__name__)

# App layout
app.layout = html.Div([
    dcc.Graph(id="scatter-plot", figure=fig),
    dcc.Tooltip(id="card-tooltip"),
])

# Callback to display the tooltip dynamically
@callback(
    Output("card-tooltip", "show"),
    Output("card-tooltip", "bbox"),
    Output("card-tooltip", "children"),
    Input("scatter-plot", "hoverData"),
)
def display_hover(hoverData):
    if hoverData is None:
        return False, no_update, no_update

    # Get the index of the hovered point
    point_index = hoverData['points'][0]['pointIndex']
    point_data = filtered_df.iloc[point_index]

    # Get data for display
    img_src = point_data['Image']
    name = point_data['Name']
    body_text = point_data['Body_Text']
    if len(body_text) > 300:
        body_text = body_text[:300] + '...'

    # Prepare the tooltip content
    children = [
        html.Div([
            html.Img(src=img_src, style={"width": "200px"}),
            html.H3(name, style={"color": "darkblue"}),
            html.P(body_text)
        ], style={'width': '250px', 'white-space': 'normal'})
    ]

    bbox = hoverData['points'][0]['bbox']
    return True, bbox, children

if __name__ == "__main__":
    app.run_server(debug=True)

<IPython.core.display.Javascript object>

In [ ]:
filtered_df

,Classifications,KeywordAbilities,Ink,Franchise,Image,Cost,Inkable,Name,Type,Lore,Unique_ID,Body_Text,Willpower,Strength,Move_Cost,Abilities
Card_Id,,,,,,,,,,,,,,,,
1_1,"Storyborn, Hero, Princess",NaN,Amber,,https://lorcana-api.com/images/ariel/on_human_legs/ariel-on_human_legs-large.png,4,True,Ariel - On Human Legs,Character,2.0,TFC-001,Voiceless - This character can't {e} to sing songs.,4.0,3.0,NaN,[Voiceless]
1_10,"Storyborn, Ally","Bodyguard, Support",Amber,,https://lorcana-api.com/images/maximus/palace_horse/maximus-palace_horse-large.png,5,True,Maximus - Palace Horse,Character,1.0,TFC-010,"Bodyguard (This character may enter play exerted. An opposing character who challenges one of your characters must choose one with Bodyguard if able.)\n\nSupport (Whenever this character quests, you may add their {s} to another chosen character's {s} this turn.)",5.0,4.0,NaN,"[Bodyguard, Support]"
1_103,"Storyborn, Ally",NaN,Ruby,,https://lorcana-api.com/images/abu/mischievous_monkey/abu-mischievous_monkey-large.png,3,True,Abu - Mischievous Monkey,Character,2.0,TFC-103,NaN,2.0,3.0,NaN,[]
1_104,"Floodborn, Hero",Shift 5,Ruby,,https://lorcana-api.com/images/aladdin/heroic_outlaw/aladdin-heroic_outlaw-large.png,7,True,Aladdin - Heroic Outlaw,Character,2.0,TFC-104,"Shift 5 (You may pay 5 {i} to play this on top of one of your characters named Aladdin.) \r \r Daring Exploit - During your turn, whenever this character banishes another character in a challenge, you gain 2 lore and each opponent loses 2 lore.",5.0,5.0,NaN,"[Shift 5, Daring Exploit]"
1_105,"Storyborn, Hero",NaN,Ruby,,https://lorcana-api.com/images/aladdin/street_rat/aladdin-street_rat-large.png,3,True,Aladdin - Street Rat,Character,1.0,TFC-105,"Improvise - When you play this character, each opponent loses 1 lore.",2.0,2.0,NaN,[Improvise]
1_106,"Storyborn, Ally, Captain",NaN,Ruby,,https://lorcana-api.com/images/captain/colonel's_lieutenant/captain-colonel's_lieutenant-large.png,5,True,Captain - Colonel's Lieutenant,Character,1.0,TFC-106,NaN,5.0,6.0,NaN,[]
1_107,"Storyborn, Villain, Pirate, Captain","Rush, Evasive, Reckless",Ruby,,https://lorcana-api.com/images/captain_hook/ruthless_pirate/captain_hook-ruthless_pirate-large.png,7,False,Captain Hook - Ruthless Pirate,Character,2.0,TFC-107,"Rush (This character can challenge the turn they're played.) \n\nYou Coward! - While this character is exerted, opposing characters with Evasive gain Reckless. (They can't quest and must challenge if able.)",5.0,5.0,NaN,"[Rush, You Coward!]"
1_108,Storyborn,NaN,Ruby,,https://lorcana-api.com/images/donald_duck/boisterous_fowl/donald_duck-boisterous_fowl-large.png,2,True,Donald Duck - Boisterous Fowl,Character,1.0,TFC-108,NaN,3.0,2.0,NaN,[]
1_109,"Dreamborn, Hero, Queen, Sorcerer",NaN,Ruby,,https://lorcana-api.com/images/elsa/ice_surfer/elsa-ice_surfer-large.png,4,True,Elsa - Ice Surfer,Character,1.0,TFC-109,"That's No Blizzard! - Whenever you play a character named Anna, ready this character. This character can't quest for the rest of this turn.",4.0,3.0,NaN,[That's No Blizzard!]


Now lets try some clustering!!!!

Lets look at the color distribution of the graphs:

In [ ]:
copy = filtered_df
color_counts = copy['Ink'].value_counts().reset_index()
color_counts.columns = ['Color', 'Count']  # Rename columns for clarity

# Create a bar plot
fig = px.bar(color_counts,
             x='Color',  # Using the new 'Color' column for x-axis
             y='Count',  # Using 'Count' column for y-axis
             labels={'Color': 'Color', 'Count': 'Count'},  # Labels for the axis
             title='Count of Different Color Cards')

# Show the plot
fig.show()

Visualizing card cost:

In [ ]:
# Histogram to visualize the distribution of card costs
fig = px.histogram(copy,
                   x='Cost',  # Assumes 'Cost' is the column with card cost information
                   nbins=30,  # Number of bins can be adjusted based on the data range and preference
                   labels={'Cost': 'Card Cost'},
                   title='Distribution of Card Costs')

# Add labels and a title for clarity
fig.update_layout(
    xaxis_title="Card Cost",
    yaxis_title="Number of Cards",
    title={
        'text': "Distribution of Card Costs",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

# Show the plot
fig.show()

# Web Scraping :(

We planned on scraping the deck websites to get good training data but it prooved to be so difficult. Most of them were dymanic or had bots which prevnted us from accessing any good decks. We had wrote a lot of code for it but it didnt make sense to keep it as it kept breaking or not accessing the tags we wanted to we decided to remove the code. We found some work arounds to generate decks instead


# Clustering on Card Description

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')

# Preprocess the text data
stop_words = set(stopwords.words('english'))
df['processed_text'] = df['Body_Text'].apply(lambda x: ' '.join([w.lower() for w in word_tokenize(str(x)) if w.isalpha() and w.lower() not in stop_words]))

# Create TF-IDF vectors
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['processed_text'])

# Apply K-means clustering
num_clusters = 30  # You can adjust this based on your data and requirements
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(tfidf_matrix)

# Assign cluster labels to each row
df['cluster'] = kmeans.labels_

# Explore the clusters
for cluster_id in range(num_clusters):
    print(f"Cluster {cluster_id}:")
    cluster_texts = df[df['cluster'] == cluster_id]['Body_Text']
    print(cluster_texts)
    print()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



Cluster 0:
32     We Are All Connected - Characters get +2{w} while here.\n\nLion Home - If you have a Prince or King character here, you pay 1{i} less to play characters.
167                                                                                  Laboratory - If you have a character here, you pay 1{i} less to play items.
415                                                             Loyal - If you have a character named Gaston in play, you pay 1 {i} less to play this character.
Name: Body_Text, dtype: object

Cluster 1:
5      NaN
10     NaN
12     NaN
22     NaN
31     NaN
47     NaN
55     NaN
56     NaN
65     NaN
78     NaN
83     NaN
87     NaN
99     NaN
103    NaN
109    NaN
117    NaN
122    NaN
126    NaN
133    NaN
140    NaN
145    NaN
151    NaN
168    NaN
170    NaN
184    NaN
185    NaN
188    NaN
202    NaN
207    NaN
210    NaN
221    NaN
230    NaN
240    NaN
243    NaN
259    NaN
262    NaN
264    NaN
272    NaN
277    NaN
284    NaN
287    NaN
297    NaN
2